<a href="https://colab.research.google.com/github/jasleenkaursandhu/Reproducing-chest-xray-report-generation-boag/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# KNN Model for Feature Extraction and Nearest Neighbor Analysis
# This notebook implements a KNN-based approach for chest X-ray report generation.
# It extracts DenseNet features from images and finds the most similar training images for each test image.

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tqdm
from collections import defaultdict
import pickle
!pip install pydicom
import pydicom  # Note: Original used 'dicom' but we'll use 'pydicom' which is the current version
from time import gmtime, strftime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.3 MB/s eta 0:00:00


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/mimic-cxr-project'
!mkdir -p {base_path}/data
!mkdir -p {base_path}/output

Mounted at /content/drive


In [4]:
# Import the report parser module
import sys
sys.path.append(f"{base_path}/modules")
from report_parser import parse_report, MIMIC_RE
print("Successfully imported report parser module")

Successfully imported report parser module


In [5]:
# Load train and test data
data_dir = os.path.join(base_path, 'data')
files_path = os.path.join(base_path, 'files')
output_dir = os.path.join(base_path, 'output')
features_dir = os.path.join(base_path, 'features')

train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

Train data shape: (2243, 3)
Test data shape: (871, 3)


In [6]:
# Map each dicom to its report identifier (study_id in our case)
# Note: Original code used 'rad_id', we'll use 'study_id' based on our data structure
report_id_column = 'study_id'
if report_id_column in train_df.columns:
    rad_lookup = dict(train_df[['dicom_id', report_id_column]].values)
    print(f"Created lookup using {report_id_column}")
else:
    print(f"Warning: {report_id_column} not found in columns: {train_df.columns.tolist()}")
    rad_lookup = {}

print("Sample of lookup dictionary:")
print(dict(list(rad_lookup.items())[:5]))

Created lookup using study_id
Sample of lookup dictionary:
{'f214e3d8-00d5e538-62592a72-37cf2660-0ec426a1': 58033516, 'c6f2ef94-1c46417f-1cdb50d7-bbae3a47-5dd45401': 57744330, 'f7a4e18f-004ac053-81024d1b-568dbb86-b20f308b': 50301279, '500bbbdf-966e91a0-474e045e-81e494ac-7c6124f7': 57457041, 'c0eb8f9c-b404b698-4b47abf9-cea216fd-27bea26f': 59435834}


In [8]:
# Path to neighbors file
neighbors_path = os.path.join(base_path, 'local_output/output', '1nn_neighbors.pkl')

# Check if neighbors file exists
if os.path.exists(neighbors_path):
    print(f"Loading neighbors from {neighbors_path}")
    with open(neighbors_path, 'rb') as f:
        neighbors = pickle.load(f)
    print(f"Loaded neighbors for {len(neighbors)} test images")
else:
    print(f"Neighbors file not found at {neighbors_path}")
    print("Please run the feature extraction and nearest neighbor search first.")

Loading neighbors from /content/drive/MyDrive/mimic-cxr-project/local_output/output/1nn_neighbors.pkl
Loaded neighbors for 1757 test images


In [9]:
# Define the path to the reports directory
files_path = os.path.join(base_path, 'files')

# Generate reports for each test image
generated_reports = {}

for pred_dicom in tqdm.tqdm(test_df.dicom_id):
    # Skip if we don't have neighbors for this test image
    if pred_dicom not in neighbors:
        print(f"Warning: No neighbors for {pred_dicom}")
        continue

    nn = neighbors[pred_dicom]

    found = False
    i = 0  # Start with the closest neighbor

    while not found and i < len(nn):
        nearest_dicom = nn[i]

        # Skip if we don't have a report ID for this training image
        if nearest_dicom not in rad_lookup:
            i += 1
            continue

        nearest_train_report_id = rad_lookup[nearest_dicom]

        # Get corresponding subject_id for the training image
        subject_row = train_df[train_df.dicom_id == nearest_dicom]
        if len(subject_row) == 0:
            i += 1
            continue

        subject_id = subject_row.iloc[0]['subject_id']

        # Construct path to the report
        subject_prefix = f"p{str(subject_id)[:2]}"
        subject_dir = f"p{subject_id}"
        study_dir = f"s{nearest_train_report_id}"
        report_file = f"{study_dir}.txt"
        report_path = os.path.join(files_path, subject_prefix, subject_dir, report_file)

        # Parse the report
        try:
            if os.path.exists(report_path):
                report = parse_report(report_path)

                # If the report has a findings section, use it
                if 'findings' in report:
                    found = True
                    generated_reports[pred_dicom] = report['findings']
        except Exception as e:
            pass

        i += 1

    if not found:
        print(f"Warning: Could not find a valid report for {pred_dicom}")

print(f"Generated reports for {len(generated_reports)}/{len(test_df)} test images")

  0%|          | 0/871 [00:00<?, ?it/s]

  1%|▏         | 12/871 [00:54<1:16:30,  5.34s/it]

  2%|▏         | 19/871 [01:21<1:04:37,  4.55s/it]

  3%|▎         | 29/871 [01:46<54:54,  3.91s/it]

  4%|▍         | 37/871 [02:10<47:26,  3.41s/it]

...

 96%|█████████▌| 835/871 [05:18<00:02, 14.43it/s]

 97%|█████████▋| 842/871 [05:19<00:02, 12.64it/s]

 98%|█████████▊| 854/871 [05:20<00:01, 13.18it/s]

100%|██████████| 871/871 [05:21<00:00,  2.71it/s]

Generated reports for 107/871 test images


In [10]:
# Save the generated reports to a TSV file
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

pred_dir = os.path.join(base_path, 'local_output/output')
os.makedirs(pred_dir, exist_ok=True)

pred_file = os.path.join(pred_dir, 'knn.tsv')
print(f"Saving predictions to {pred_file}")

with open(pred_file, 'w') as f:
    print('dicom_id\tgenerated', file=f)
    for dicom_id, generated in sorted(generated_reports.items()):
        # Clean up the text (remove any tabs)
        cleaned_text = generated.replace('\t', ' ')
        print(f'{dicom_id}\t{cleaned_text}', file=f)

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2025-04-21 01:11:57
Saving predictions to /content/drive/MyDrive/mimic-cxr-project/local_output/output/knn.tsv
2025-04-21 01:11:57


In [11]:
# Display sample of generated reports
sample_count = min(3, len(generated_reports))
sample_dicoms = list(generated_reports.keys())[:sample_count]

for dicom_id in sample_dicoms:
    print(f"\nSample report for {dicom_id}:")
    report_text = generated_reports[dicom_id]

    # Print preview of the report (first 200 characters)
    if len(report_text) > 200:
        print(report_text[:200] + "...")
    else:
        print(report_text)


Sample report for 6497aa38-6297f53a-fa4befd4-ba1a4958-64aedec6:
tracheostomy tube again noted. ng tube again noted, extending beneath the diaphragm to overlie the stomach. right subclavian picc line tip lies near the svc/ ra junction, similar to prior. cardiomedia...

Sample report for 814416f0-c76e5d61-9a01f44f-4d5ebd41-87097577:
the et tube is no longer visualized. spinal fixation device and ng tube are present. there is complete opacification of the right lower lobe compatible with a combination of infiltrate and volume loss...

Sample report for 001bb54b-a4e0bb99-48a28f4c-9df85f1b-e1606587:
left picc tip terminates in the lower svc, unchanged. right-sided dual-lumen central venous catheter terminates in the right atrium, unchanged. tracheostomy tube tip remains in stable position. the pa...
